# 复现经典网络：LeNet5与AlexNet

## 1.现代CNN的奠基者：LeNet5

使用卷积层和池化层能够创建的最简单的网络是什么样的呢？或许就是下面这样的架构：

![Alt text](image-122.png)

首先，图像从左侧输入，从右侧输出，数据传输方向与DNN一致。整个网络由2个卷积层，2个平均池化层和2个全连接层组成，虽然没有标注出来，但每个卷积层和全连接层后都使用激活函数tanh或sigmoid。

这个架构就是著名的LeNet4架构，他在1988年被LeCun等人在论文《Gradient-Based Learning Applied to Document Regognition》中正式提出，它被认为是现代卷积神经网络的奠基者。在LeNet5被提出后，几乎所有的卷积网络都会连用卷积层，池化层与全连接层（也就是线性层）。现在，这已经成为了一种非常经典的架构：卷积层作为输入层，紧跟激活函数，池化层后紧跟一个或数个卷积+激活的结构之后。在卷积池化交替进行数次之后，转向线性层+激活函数，并使用线性层结尾，输出预测结果。由于输入数据结构的设置，以上架构图中的网络可能与论文中有一些细节上的区别，在其他教材中，你或会见到添加了更多卷积层，或添加了更多池化层的相似架构，这些都是根据LeNet的核心思想“卷积+池化+线性”来搭建的LeNet4的变体。

在上面的架构中，每层的结构和参数都标识得非常清楚，在PyTorch中实现其架构的代码如下：

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

data = torch.ones(size= (10, 1, 32, 32))

# 定义网络
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
    
    def forward(self, x):   # tanh作为激活函数
        x = self.conv1(x)
        x = F.tanh(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.tanh(x)
        x = self.pool2(x)
        x = x.view(-1, 16*5*5)
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x
    

In [2]:
# 实例化网络
net = LeNet()
# 前向传播
net(data)   # 相当于在执行net.forward(data)

tensor([[0.0111, 0.0125, 0.0124, 0.0132, 0.0119, 0.0131, 0.0143, 0.0116, 0.0105,
         0.0087, 0.0113, 0.0143, 0.0130, 0.0130, 0.0108, 0.0126, 0.0148, 0.0114,
         0.0111, 0.0108, 0.0132, 0.0114, 0.0106, 0.0114, 0.0138, 0.0100, 0.0113,
         0.0100, 0.0130, 0.0119, 0.0119, 0.0137, 0.0110, 0.0113, 0.0129, 0.0143,
         0.0102, 0.0098, 0.0120, 0.0124, 0.0112, 0.0133, 0.0111, 0.0115, 0.0120,
         0.0124, 0.0121, 0.0130, 0.0127, 0.0102, 0.0122, 0.0121, 0.0113, 0.0109,
         0.0126, 0.0126, 0.0126, 0.0119, 0.0116, 0.0111, 0.0127, 0.0099, 0.0110,
         0.0118, 0.0123, 0.0100, 0.0112, 0.0124, 0.0117, 0.0104, 0.0137, 0.0134,
         0.0098, 0.0135, 0.0093, 0.0114, 0.0119, 0.0141, 0.0133, 0.0112, 0.0111,
         0.0117, 0.0120, 0.0134],
        [0.0111, 0.0125, 0.0124, 0.0132, 0.0119, 0.0131, 0.0143, 0.0116, 0.0105,
         0.0087, 0.0113, 0.0143, 0.0130, 0.0130, 0.0108, 0.0126, 0.0148, 0.0114,
         0.0111, 0.0108, 0.0132, 0.0114, 0.0106, 0.0114, 0.0138, 0.0100, 0.

In [4]:
# torchinfo
!pip install torchinfo

  Obtaining dependency information for torchinfo from https://files.pythonhosted.org/packages/72/25/973bd6128381951b23cdcd8a9870c6dcfc5606cb864df8eabd82e529f9c1/torchinfo-1.8.0-py3-none-any.whl.metadata


In [5]:
from torchinfo import summary
summary(net, input_size=(10, 1, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
LeNet                                    [10, 84]                  --
├─Conv2d: 1-1                            [10, 6, 28, 28]           156
├─AvgPool2d: 1-2                         [10, 6, 14, 14]           --
├─Conv2d: 1-3                            [10, 16, 10, 10]          2,416
├─AvgPool2d: 1-4                         [10, 16, 5, 5]            --
├─Linear: 1-5                            [10, 120]                 48,120
├─Linear: 1-6                            [10, 84]                  10,164
Total params: 60,856
Trainable params: 60,856
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 4.22
Input size (MB): 0.04
Forward/backward pass size (MB): 0.52
Params size (MB): 0.24
Estimated Total Size (MB): 0.81

这是我们在之前的课程中学到的写法，这种写法适合层数较少，层次简单的神经网络，但这个网络和我们之前写的只有线性层的网络比起来还是略有些复杂了。在这段代码中你可以清晰的看见卷积层，池化层以及全连接层是如何连接在一起共同作用的。我们将“层”放在init中定义，而将函数放在forward中定义，令init中的结构与forward中的结构一一对应，并使用forward函数执行向前传播流程。

从今天的眼光来看，LeNet5无疑是很简单并且有些弱小的卷积网络，然而，当加入学习绿衰减等优化方法并训练恰当时，但一的LeNet5模型可以在Fashion-MINIST数据集上获得超过91%的训练准确率。这个优化效果已经比只有线性层的网络提升了约5%的成绩。虽然简单，LeNet5却切实展示了卷积网络的实力。